<a href="https://colab.research.google.com/github/baiyinnamula/-Pandas-/blob/main/%E7%AC%AC10%E7%AB%A0Pandas%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第10章Pandas数据清洗

数据清洗是数据分析的一个重要步骤，关系到数据的质量，而数据
的质量又关系到数据分析的效果。数据清洗一般包括缺失值填充、冗余
数据删除、数据格式化、异常值处理、逻辑错误数据检测、数据一致性
校验、重复值过滤、数据质量评估等。

In [ ]:
import pandas as pd
import numpy as np

## 10.1 缺失值的认定

### 10.1.1 缺失值类型

一般使用特殊类型NaN代表缺失值，可以用NumPy定义为np.NaN或
np.nan。在Pandas 1.0以后的版本中，实验性地使用标量pd.NA来代表。

In [ ]:
df = pd.DataFrame({
'A':['a1', 'a1', 'a2', 'a2'],
'B':['b1', 'b2', None, 'b2'],
'C':[1, 2, 3, 4],
'D':[5, 6, None, 8],
'E':[5, None, 7, 8]
})
df

In [ ]:
# 将无穷值设置为缺失值
pd.options.mode.use_inf_as_na = True

### 10.1.2 缺失值判断
df.isna()及其别名df.isnull()是Pandas中判断缺失值的主要方法。对整
个数据进行缺失值判断，True为缺失。

In [ ]:
df.isna()

In [ ]:
df.isnull()

In [ ]:
# 对某一列进行检测
df.D.isna()

相反地，df.notna()可以让非缺失值显示为True，让缺失值显示为
False

In [ ]:
df.notna()

In [ ]:
# 检测某一列非缺失值
df.D.notna()

### 10.1.3 缺失值统计

如果需要统计一个数据中有多少个缺失值，可利用sum计算，计算
时将False当作0、将True当作1的特性。


In [ ]:
# 每一列有多少缺失值
df.isna().sum()

In [ ]:
# 每行有多少缺失值
df.isna().sum(axis=1)

In [ ]:
# 总共有多少缺失值
df.isna().sum().sum()

### 10.1.4 缺失值筛选

In [ ]:
df

In [ ]:
# 有缺失值的行

df.loc[df.isna().any(1)]

In [ ]:
# 有缺失值的列

df.loc[:, df.isna().any()]

In [ ]:
# 没有缺失值的行
df.loc[~(df.isna().any(1))]

In [ ]:
# 没有缺失值的列
df.loc[:, ~(df.isna().any())]

In [ ]:
pd.Series([True, True, False]).sum()

### 10.1.5 NA标量

Pandas 1.0以后的版本中引入了一个专门表示缺失值的标量pd.NA，
它代表空整数、空布尔、空字符，这个功能目前处于实验阶段。pd.NA
的目标是提供一个“缺失值”指示器，该指示器可以在各种数据类型中一
致使用（而不是np.nan、None或pd.NaT，具体取决于数据类型）。

In [ ]:
s = pd.Series([1, 2, None, 4], dtype="Int64")

In [ ]:
s

In [ ]:
s[2] is pd.NA

In [ ]:
pd.isna(pd.NA)

In [ ]:
pd.NA + 1

### 10.1.6 时间数据中的缺失值

对于时间数据中的缺失值，Pandas提供了一个NaT来表示，并且
NaT和NaN是兼容的。

In [ ]:
pd.Series([pd.Timestamp('20200101'), None, pd.Timestamp('20200102')])

### 10.1.7 整型数据中的缺失值

由于NaN是浮点型，因此缺少一个整数的列可以转换为整型。

In [ ]:
type(df.at[2, 'D'])

In [ ]:
pd.Series([1, 2, np.nan], dtype=pd.Int64Dtype())

### 10.1.8 插入缺失值

In [ ]:
# 修改为缺失值
df.loc[0] = None
df.loc[1] = np.nan
df.A = pd.NA
df

### 10.1.9 小结

## 10.2 缺失值的操作

对于缺失值，我们通常会根据业务需要进行修补，但对于缺失严重
的数据，会直接将其删除。

### 10.2.1 缺失值填充

In [ ]:
# 原数据
df = pd.DataFrame({
'A':['a1', 'a1', 'a2', 'a2'],
'B':['b1', 'b2', None, 'b2'],
'C':[1, 2, 3, 4],
'D':[5, 6, None, 8],
'E':[5, None, 7, 8]
})
df

对于缺失值，我们常用的一个办法是利用一定的算法去填充它。这
样虽然不是特别准确，但对于较大的数据来说，不会对结果产生太大影
响。df.fillna(x)可以将缺失值填充为指定的值。


In [ ]:
df.fillna(0)

常用的方法有以下几个：

In [ ]:
# 填充为 0
df.fillna(0)
# 填充为指定字符
df.fillna('missing')
df.fillna('暂无')
df.fillna('待补充')
# 指定字段填充
df.A.fillna('暂无')
# 指定字段填充
df.A.fillna(0, inplace=True)
# 只替换第一个
df.fillna(0, limit=1)
# 将不同列的缺失值替换为不同的值
values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
df.fillna(value=values)

有时候我们不能填入固定值，而要按照一定的方法填充。df.fillna()
提供了一个method参数，可以指定以下几个方法。

* pad / ffill：向前填充，使用前一个有效值填充，
df.fillna(method='ffill')可以简写为df.ffill()。
* bfill / backfill：向后填充，使用后一个有效值填充，
df.fillna(method='bfill')可以简写为df.bfill()。

In [ ]:
df = pd.DataFrame({
'A':['a1', 'a1', 'a2', 'a2'],
'B':['b1', 'b2', None, 'b2'],
'C':[1, 2, 3, 4],
'D':[5, 6, None, 8],
'E':[5, None, 7, 8]
})
df

In [ ]:
# 取后一个有效值填充
df.fillna(method='bfill')

In [ ]:
# 取前一个有效值填充
df.fillna(method='ffill')

除了取前后值，还可以取经过计算得到的值，比如常用的平均值填
充法：

In [ ]:
# 填充列的平均值
df.fillna(df.mean())
# 对指定列填充平均值
df.fillna(df.mean()['B':'C'])
# 另一种填充列的平均值的方法
df.where(pd.notna(df), df.mean(), axis='columns')

缺失值填充的另一个思路是使用替换方法df.replace()

In [ ]:
df = pd.DataFrame({
'A':['a1', 'a1', 'a2', 'a2'],
'B':['b1', 'b2', None, 'b2'],
'C':[1, 2, 3, 4],
'D':[5, 6, None, 8],
'E':[5, None, 7, 8]
})
df

In [ ]:
df.replace({'B':{np.nan: 100}})

### 10.2.2 插值填充

插值（interpolate）是离散函数拟合的重要方法，利用它可根据函
数在有限个点处的取值状况，估算出函数在其他点处的近似值。Series
和DataFrame对象都有interpolate()方法，默认情况下，该方法在缺失值
处执行线性插值。它利用数学方法来估计缺失点的值，对于较大的数据
非常有用。

In [ ]:
s = pd.Series([0, 1, np.nan, 3])

In [ ]:
s.interpolate()

其中默认method ='linear'，即使用线性方法，认为数据呈一条直
线。method方法指定的是插值的算法。

* 如果你的数据增长速率越来越快，可以选择method='quadratic'二次
插值；
* 如果数据集呈现出累计分布的样子，推荐选择method='pchip'；

* 如果需要填补默认值，以平滑绘图为目标，推荐选择method='akima'。这些都需要你的环境中安装了SciPy库。

### 10.2.3 缺失值删除

如果数据对完整性要求比较高，只要有缺失值，就会认为数据是无
效的。比如一份问卷的回答比例过低，那么就认为它是无效的，就需要
整行整列删除。我们使用df.dropna()方法来删除缺失值。

In [ ]:
df = pd.DataFrame({
'A':['a1', 'a1', 'a2', 'a2'],
'B':['b1', 'b2', None, 'b2'],
'C':[1, 2, 3, 4],
'D':[5, 6, None, 8],
'E':[5, None, 7, 8]
})
df

In [ ]:
# 删除有缺失值的行

df.copy().dropna()

In [ ]:
# 删除有缺失值的列
df.copy().dropna(1)

### 10.2.4 缺失值参与计算

In [ ]:
df

In [ ]:
df.sum()

In [ ]:
df.D.cumsum()

cumsum()和cumprod()会忽略NA值，但值会保留在序列中，可以使
用skipna=False跳过有缺失值的计算并返回缺失值。


In [ ]:
df.D.cumsum(skipna=False)

In [ ]:
# df.count()在统计时，缺失值不计数
df.count()

In [ ]:
df

In [ ]:
# 如果聚合分组的列里有空值，则会自动忽略这些值（当它不存在）

df.groupby('B').sum()

In [ ]:
df

In [ ]:
# 聚合计入缺失值
df.groupby('B', dropna=False).sum()

### 10.2.5 小结

本节介绍了缺失值的填充方法。如果数据质量有瑕疵，在不影响分
析结果的前提下，可以用固定值填充、插值填充。对于质量较差的数据可以直接丢弃。

## 10.3 数据替换

Pandas中数据替换的方法包含数值、文本、缺失值等替换，经常用
于数据清洗与整理、枚举转换、数据修正等情形。

Series和DataFrame中的replace()都提供了一种高效而灵活的方法。

### 10.3.1 指定值替换

In [ ]:
ser = pd.Series([0., 1., 2., 3., 4.])
ser

In [ ]:
ser.replace(0, 5)

### 10.3.2 使用替换方式

In [ ]:
# 将 1,2,3 替换为它们前一个值
ser.replace([1, 2, 3], method='pad')

In [ ]:
# 将 1,2,3 替换为它们前一个值
ser.replace([1, 2, 3], method='bfill')

### 10.3.3 字符替换

In [ ]:
# 把bat替换为new，不使用正则表达式
df.replace(to_replace='bat', value='new')
# 利用正则表达式将ba开头的值替换为new
df.replace(to_replace=r'^ba.$', value='new', regex=True)
# 如果多列规则不一，可以按以下格式对应传入
df.replace({'A': r'^ba.$'}, {'A': 'new'}, regex=True)
# 多个规则均替换为同样的值
df.replace(regex=[r'^ba.$', 'foo'], value='new')
# 多个正则及对应的替换内容
df.replace(regex={r'^ba.$': 'new', 'foo': 'xyz'})


### 10.3.4 缺失值替换

In [ ]:
d = {'a':list(range(4)), 'b': list('ab..')}
d

In [ ]:
df = pd.DataFrame(d)

In [ ]:
df

In [ ]:
df = df.replace('.', np.nan)
df

In [ ]:
df.replace(['a', '.'], ['b', np.nan])

### 10.3.5 数字替换

In [ ]:
# 生成数据
df = pd.DataFrame(np.random.randn(10, 2))
df[np.random.rand(df.shape[0]) > 0.5] = 1.5
df

In [ ]:
df.replace(1.5, np.nan)

### 10.3.6 数据修剪

对于数据中存在的极端值，过大或者过小，可以使用df.clip(lower,upper)来修剪。
当数据大于upper时使用upper的值，小于lower时用lower的值，这和numpy.clip方法一样。

In [ ]:
# 包含极端值的数据
df = pd.DataFrame({'a': [-1, 2, 5], 'b': [6, 1, -3]})
df

In [ ]:
df.clip(0, 3)

In [ ]:
c = pd.Series([-1, 1, 3])
df.clip(c, c+1, axis=0)

### 10.3.7 小结

替换数据是数据清洗的一项很普遍的操作，同时也是修补数据的一种有效方法。df. replace()方法功能强大，在本节中，我们了解了它实现定值替换、定列替换、广播替换、运算替换等功能

## 10.4 重复值及删除数据

数据在收集、处理过程中会产生重复值，包括行和列，既有完全重
复，又有部分字段重复。重复的数据会影响数据的质量，特别是在它们参与统计计算时。

### 10.4.1 重复值识别
df.duplicated()是Pandas用来检测重复值的方法。

它可以返回表示重复行的布尔值序列，默认为一行的所有内容，
subset可以指定列。keep参数用来确定要标记的重复值，可选的值有：
* first：将除第一次出现的重复值标记为True，默认。
* last：将除最后一次出现的重复值标记为True。
* False：将所有重复值标记为True。


In [ ]:
# 原数据
df = pd.DataFrame({
'A': ['x', 'x', 'z'],
'B': ['x', 'x', 'x'],
'C': [1, 1, 2]
})
df

In [ ]:
# 全行检测，除第一次出现的外，重复的为True
df.duplicated()

In [ ]:
# 除最后一次出现的外，重复的为True
df.duplicated(keep='last')

In [ ]:
# 所有重复的都为True
df.duplicated(keep=False)

In [ ]:
# 指定列检测
df.duplicated(subset=['B'], keep=False)

In [ ]:
# 筛选出重复内容
df[df.duplicated()]

### 10.4.2 删除重复值

In [ ]:
# 删除重复值语法
df.drop_duplicates(subset=None,
keep='first',
inplace=False,
ignore_index=False)

In [ ]:
df.drop_duplicates()

In [ ]:
df.drop_duplicates(subset=['A'])

In [ ]:
# 保留最后一个
df.drop_duplicates(subset=['A'], keep='last')

### 10.4.3 删除数据

df.drop()通过指定标签名称和相应的轴，或直接给定索引或列名称
来删除行或列。使用多层索引时，可以通过指定级别来删除不同级别上的标签。

In [ ]:
# 语法
df.drop(labels=None, axis=0,
  index=None, columns=None,
  level=None, inplace=False,
  errors='raise')

其中的参数说明如下：
* labels：要删除的列或者行，如果要删除多个，传入列表。
* axis：轴的方向，0为行，1为列，默认为0。
* index：指定的一行或多行。
* column：指定的一列或多列。
* level：索引层级，将删除此层级。
* inplace：布尔值，是否生效。
* errors：ignore或者raise，默认为raise，如果为ignore，则容忍错误，仅删除现有标签。

### 10.4.4 小结
本节介绍了三个重要的数据清洗工具：
* df.duplicated()能够识别出重复值，返回一个布尔序列，用于查询和筛选重复值；
* df.drop_duplicates() 可以直接删除指定的重复数据；
* df.drop()能够灵活地按行或列删除指定的数据，可以通过计算得到异常值所在的列和行再执行删除。

## 10.5 Numpy 格式转换

### 10.5.1 转换方法

Pandas 0.24.0引入了两种从Pandas对象中获取NumPy数组的新方
法。
* ds.to_numpy()：可以用在Index、Series和DataFrame对象；
* s.array：为PandasArray，用在Index和Series，它封装了
numpy.ndarray接口。

### 10.5.2 DataFrame转为ndarray

In [ ]:
# df.values # 不推荐
df.to_numpy()

### 10.5.3 Series转为ndarray

In [ ]:
ser.to_numpy()

### 10.5.4 df.to_records()

可以使用to_records()方法，但是如果数据类型不是你想要的，则必须对它们进行一些处理。

### 10.5.5 np.array读取

可以用np.array直接读取DataFrame或者Series数据，最终也会转换为array类型。

In [ ]:
np.array(df)

In [ ]:
np.array(df.A)

### 10.5.6 小结
本节介绍了如何将Pandas的两大数据类型DataFrame和Series转为
NumPy的格式，推荐使用to_numpy()方法。

## 10.6 本章小结

  数据清洗是我们获取到数据集后要做的第一件事，处理缺失数据和缺失值是数据清洗中最棘手的部分。
    
  只有保证数据的高质量才有可能得出高质量的分析结论，一些数据建模和机器学习的场景对数据质量有严格的要求，甚至不允许有缺失值。

  本章介绍了在Pandas中缺失值的表示方法以及如何找到缺失值，重复值的筛选方法以及如何对它们进行删除、替换和填充等操作。
  
  完成这些工作，将得到一个高质量的数据集，为下一步数据分析做好准备。